<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-HW-4/blob/main/ML_Zoomcamp_HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning ZoomCamp - Homework 3 
**Author.....: Carlos Manuel de Oliveira Alves**<br>
**Created...: 02/10/2022**

In [7]:
# Import all necessay libraries that we will use in this homework
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold


from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.linear_model import Ridge
from sklearn import metrics

# Import the library warnings to ignore the warnings from the system
import warnings
warnings.filterwarnings("ignore")

In [8]:
# Data Import:

# Locate dataset for the homework
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv'

In [9]:
# Download dataset and store it locally with different file name
!wget $data -O data.csv

--2022-10-02 13:00:33--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73250 (72K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  71.53K  --.-KB/s    in 0.008s  

2022-10-02 13:00:34 (8.99 MB/s) - ‘data.csv’ saved [73250/73250]



In [18]:
# Store data using the Pandas library for this homework
data = pd.read_csv('data.csv')

In [19]:
# Check total of rows and columns of the dataset
print(data.shape)

(1319, 12)


In [12]:
# Check the first 10 rows of the dataset
data.head(10)

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,yes,0,37.66667,4.5200,0.033270,124.983300,yes,no,3,54,1,12
1,yes,0,33.25000,2.4200,0.005217,9.854167,no,no,3,34,1,13
2,yes,0,33.66667,4.5000,0.004156,15.000000,yes,no,4,58,1,5
3,yes,0,30.50000,2.5400,0.065214,137.869200,no,no,0,25,1,7
4,yes,0,32.16667,9.7867,0.067051,546.503300,yes,no,2,64,1,5
5,yes,0,23.25000,2.5000,0.044438,91.996670,no,no,0,54,1,1
6,yes,0,27.91667,3.9600,0.012576,40.833330,no,no,2,7,1,5
7,yes,0,29.16667,2.3700,0.076434,150.790000,yes,no,0,77,1,3
8,yes,0,37.00000,3.8000,0.245628,777.821700,yes,no,0,97,1,6
9,yes,0,28.41667,3.2000,0.019780,52.580000,no,no,0,65,1,18


In [20]:
# Create the target variable by mapping yes to 1 and no to 0
df.target = df.target.map(target_variables)



AttributeError: ignored